### Import function

In [9]:
from bs4 import BeautifulSoup as soup
import requests
from datetime import datetime
import pandas as pd
import json
from dsmlibrary.datanode import DataNode

### Upload web ที่ทำการ Scrapy และทำการ Scrapy ข้อมูล

In [10]:
r = requests.get('https://thestandard.co/?s=ขมิ้นชัน&search=')
b = soup(r.content,'html.parser')

In [11]:
title = []
urls = []
content = []
published_date = []
retrieved_date  = []

In [12]:
for news in b.findAll('div', attrs={'class' : 'col-md-8'}):
    titles = news.find('h3').get_text()
    titles = titles.replace('\n','')
    urls.append(news.find('a', href=True)['href'])
    #print(titles)
    title.append(titles)

#print(title)
#print(urls)

In [13]:
for link in urls:
    page = requests.get(link)
    bsobj = soup(page.content,'html.parser')
    for n in bsobj.findAll('div', attrs={'class' : 'entry-content'}):
        dis = n.text
        dis = dis.replace('\n','')
        dis = dis.replace('\xa0','')
        content.append(dis)

    for date in bsobj.findAll('div', attrs={'class' : 'meta-date'}):
        date_p = date.find('span').get_text()
        date_p = datetime.strptime(date_p, '%d.%m.%Y').strftime("%Y-%m-%d")
        published_date.append(date_p)
        
    r = (datetime.now()).strftime("%Y-%m-%d")
    retrieved_date.append(r)

In [14]:
#สร้าง Dataframe เพื่อให้ข้อมูลเป็นสัดส่วน
table = pd.DataFrame([title, published_date, retrieved_date, content, urls]).transpose()
table.columns = ['title','publish_date','retrieve_date','content','source_url']
table.head()

,title,publish_date,retrieve_date,content,source_url
0,แพทย์เผย ยังไม่มีงานวิจัยยืนยัน จิบน้ำขมิ้นชัน...,2020-07-30,2022-11-15,วันนี้ (30 กรกฎาคม) นพ.สมศักดิ์ อรรฆศิลป์ อธิบ...,https://thestandard.co/physician-say-no-eviden...
1,10 ประโยชน์ของขมิ้นชันต่อสุขภาพผิวหนัง,2019-07-02,2022-11-15,ขมิ้น หรือขมิ้นชัน จัดเป็นพืชสมุนไพรที่คนไทยผู...,https://thestandard.co/turmeric-benefits-for-s...
2,เชฟวิชิต แห่ง Royal Osha พร้อมเสิร์ฟเมนูใหม่สไ...,2022-10-24,2022-11-15,ใครที่เป็นแฟนอาหารฝีมือเชฟมิชลินสตาร์ 1 ดาวอย่...,https://thestandard.co/royal-osha-chefs-table-...
3,เปิดประสบการณ์นวดฟินกาย สบายใจ ที่ Vana Belle ...,2022-09-16,2022-11-15,หากคุณกำลังมีแพลนจะไปเยือนเกาะสมุย และกำลังมอง...,https://thestandard.co/vana-belle-koh-samui/
4,ส่องเทรนด์ Wellness Tourism การท่องเที่ยวเชิงส...,2022-06-03,2022-11-15,การระบาดของเชื้อไวรัสและโรคภัยต่างๆ ในปัจจุบัน...,https://thestandard.co/wellness-tourism/


In [15]:
table = table.to_dict('records')

### เรียกใช้ datanode และUpload ข้อมูลลง DataPlatform

In [16]:
with open('turmeric_thestandard.json','w', encoding='utf8') as json_file:
    json.dump(table, json_file)

In [17]:
dir_raw_id = 71

In [18]:
datanode = DataNode()

Init DataNode sucessful!


In [19]:
datanode.upload_file(directory_id=dir_raw_id, file_path='turmeric_thestandard.json')

File turmeric_thestandard.json alrady exists, do you want to replace y/n :  y


  0%|          | 0.00/182k [00:00<?, ?B/s]

{'id': 830,
 'msg': 'file turmeric_thestandard.json create sucessful',
 'path': 'kanlaya/DSI310-17/rawData/turmeric_thestandard.json',
 'uploaded': True}